In [1]:
import numpy as np
import pandas as pd
import scipy.sparse

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import TruncatedSVD
import string
from os import listdir
import os
import errno
import subprocess
from collections import defaultdict
import operator
import re
import pickle
from collections import OrderedDict
import six # needed for Google Cloud client

!pip install unidecode
import unidecode

#!pip install --upgrade --ignore-installed google-cloud-language
# Imports the Google Cloud client library
from google.cloud import language
from google.cloud.language import enums
from google.cloud.language import types

In [2]:
def translate_non_alphanumerics(to_translate, translate_to=u''):
    # https://stackoverflow.com/a/1324274/2491761
    not_letters = u'!"#%\'()&*+,-./:;<=>?@[\]^_`{|}~0123456789'
    if isinstance(to_translate, unicode):
        #print "unicode instance"
        translate_table = dict((ord(char), unicode(translate_to)) for char in not_letters)
    else:
        #print "not unicode instance"
        assert isinstance(to_translate, str)
        #translate_table = string.maketrans(not_letters, translate_to *len(not_letters))
        return to_translate.translate(None, string.punctuation + string.digits)
    return to_translate.translate(translate_table)

def extract_ingredients(full_ingred_list, strip_punct=True):
    # From: https://en.wikibooks.org/wiki/Cookbook:Units_of_measurement
    meas_units = [
        'teaspoon', 'teaspoons',
        't',
        'tsp', 'tsps',
        'tablespoon','tablespoons',
        'tbl',
        'tbs',
        'tbsp', 'tbsps',
        'fl', 'fluid',
        'oz', 'ozs',
        'ounce', 'ounces',
        'cup', 'cups',
        'c',
        'pint', 'pints',
        'p',
        'pt',
        'quart', 'quarts',
        'qt', 'qts',
        'q', 'qs',
        'gallon', 'gallons',
        'gal', 'gals',
        'ml',
        'milliliter', 'milliliters',
        'millilitre', 'millilitres',
        'cc',
        'l',
        'liter', 'liters',
        'litre', 'litres',
        'pinch', 'pinches',        
        'pound', 'pounds',
        'lb', 'lbs',
        'mg', 'mgs',
        'milligram', 'milligrams',
        'milligramme', 'milligrammes',
        'g', 'gs',
        'gram', 'grams',
        'gramme', 'grammes',
        'kg', 'kgs',
        'kilogram', 'kilograms',
        'kilogramme', 'kilogrammes',
        'mm', 'mms',
        'millimeter', 'millimeters',
        'millimetre', 'millimetres',
        'cm', 'cms',
        'centimeter', 'centimeters',
        'centimetre', 'centimetres',
        'm', 'ms',
        'meter', 'meters',
        'metre', 'metres',
        'inch', 'inches',
        'in', 'ins',
        ###
        'loaf', 'loaves',
        'pouch', 'pouches',
        'wedge', 'wedges',
        'drop', 'drops',
        'amount', 'amounts',
        'bulk', 'bulks',
        'coating', 'coatings',
        'carton', 'cartons',
        'count',
        'dusting', 'dustings',
        'roll', 'rolls',
        'hint', 'hints',
        'round', 'rounds',
        'cube', 'cubes',
        'husk', 'husks',
        'envelope', 'envelopes',
        'container', 'containers',
        'dash', 'dashes',
        'bitesize', 'bitesized',
        'bite', 'sized',
        'size',
        'each',
        'taste', 'desired',
        'can', 'cans',
        'unit', 'units',
        'box', 'boxes',
        'tub', 'tubs',
        'slab', 'slabs',
        'sprig', 'sprigs',
        'stalk', 'stalks',
        'matchstick', 'matchsticks',
        'balls',
        'clove', 'cloves',
        'slice', 'slices',
        'head', 'heads',
        'spear', 'spears',
        'chunk', 'chunks',
        'piece', 'pieces',
        'jar', 'jars',
        'package', 'packages',
        'pack', 'packs',
        'packet', 'packets',
        'bunch', 'bunches',
        'tube', 'tubes',
        'jug', 'jugs',
        'bottle', 'bottles',
        'stick', 'sticks',
        'strip', 'strips',
        'bag', 'bags',
        'dash', 'dashes',
        'container', 'containers',
        'envelope', 'envelopes',
        'rounds',
        'sheet', 'sheets',
        'squares',
        'whole',
        'extra', 'extras',
        'dozen', 'dozens',
        'half', 'halves',
        'third', 'thirds',
        'quarter', 'quarters',
        'fifth', 'fifths',
        'eighth', 'eighths',
    ]
    
    preparatory_descriptions = [
        'baked',
        'beat', 'beaten', 'wellbeaten',
        'blanched',
        'blended',
        'boiled',
        'bottled',
        'broiled',
        'broken',
        'browned',
        'canned',
        'chilled',
        'chipped',
        'chopped',
        'cleaned',
        'converted',
        'cooked',
        'cooled',
        'cored',
        'crumbled',
        'crushed',
        'cubed',
        'cut',
        'defrosted',
        'deshelled',
        'desilked',
        'deveined',
        'diced', 'dice',
        'dissolved',
        'divided',
        'drained',
        'dried',
        'filleted',
        'filtered',
        'flaked',
        'fresh',
        'fried',
        'frozen',
        'grated',
        'grilled',
        'ground',
        'halved',
        'hardened',
        'heated',
        'hulled',
        'husked',
        'jarred',
        'juiced',
        'julienned', 'julienne', 'juliennecut', 'juliennesliced',
        'kneaded', 'kneading',
        'marinated',
        'mashed',
        'matchstickcut',
        'melted',
        'microwaved',
        'minced',
        'mixed',
        'opened',
        'packed',
        'peeled',
        'pitted',
        'prepared',
        'pressed',
        'prewashed',
        'processed',
        'quartered',
        'raw',
        'refrigerated',
        'removed',
        'rinsed',
        'ripe',
        'ripened',
        'ripped',
        'roasted',
        'rolled', 'rolling',
        'salted',
        'saved',
        'scored',
        'scrubbed',
        'seasoned',
        'sectioned',
        'seeded',
        'separated',
        'shaved',
        'shelled',
        'shredded',
        'shucked',
        'sifted',
        'smashed',
        'smoked',
        'snipped',
        'sliced',
        'slivered',
        'softened',
        'split',
        'squeezed',
        'steamed',
        'stemmed',
        'soaked',
        'sweetened',
        'thawed',
        'toasted',
        'torn',
        'trimmed',
        'uncored',
        'uncooked',
        'undrained',
        'unhusked',
        'unopened',
        'unpeeled',
        'unsalted',
        'unsweetened',
        'unwashed',
        'use', 'used',
        'warmed',
        'washed',
        'zested',
    ]
    
    other_words_to_filter = [
        # articles:
        'a', 'an', 'the',
        # adverbs:
        'all',
        'approximately',
        'coarsely',
        'crosswise',
        'diagonally', 
        'finely',
        'freshly',
        'horizontally',
        'left',
        'lengthwise', 
        'lightly',
        'like',
        'overnight',
        'piecewise',
        'plus',
        'right',
        'roughly',
        'sideways',
        'slightly',
        'stiffly',
        'then',
        'thickly',
        'thinly',
        'vertically',
        'very',
        'widthwise', 
        # adjectives:
        'according',
        'allpurpose',
        'another',
        'any',
        'certified',
        'classic',
        'coarse', 
        'cold',
        'discarded',
        'dry',
        'extralarge',
        'extrasmall', 
        'fine', 
        'firm',
        'fresh',
        'hard',
        'hot',
        'jumbo',
        'large', 
        'lean',
        'leftover',
        'less',
        'long',
        'medium',
        'more',
        'multicolored',
        'natural',
        'needed',
        'only',
        'optional',
        'organic',
        'original',
        'other',
        'pure',
        'real',
        'rough',
        'short',
        'small',
        'stale',
        'soft',
        'such',
        'thick',
        'thickcut',
        'thicklycut',
        'thicklysliced',
        'thicksliced',
        'thin',
        'thincut',
        'thinlycut',
        'thinlysliced',
        'thinsliced',
        # prepositions:
        'about',
        'across',
        'against',
        'apart',
        'as',
        'at',
        'by',
        'for',
        'from',
        'in',
        'into',
        'of',
        'on',
        'to',
        'with',
        'without',
        # conjunctions:
        'and',
        'if',
        'or',
        # abbreviations:
        'eg', 'ie',
        # nouns:
        'directions',
        'hand',
        'ingredients',
        'list',
        'purpose',
        'room',
        'temperature',
        # verbs:
        'add',
        'including',
    ]
    
    #print "Before..."
    #print full_ingred_list
    
    # strip punctuation chars
    # all lowercase
    if strip_punct:
        cleaned_ingred_list = [translate_non_alphanumerics(s).lower() for s in full_ingred_list]
    else:
        cleaned_ingred_list = full_ingred_list
    #print cleaned_ingred_list
    
    # split on whitespace
    # throw out meas units
    #print
    #print "After..."
    if isinstance(cleaned_ingred_list[0], unicode):
        ingreds_only = [' '.join(w for w in unidecode.unidecode(ingred_item).split() if w not in meas_units and w not in preparatory_descriptions and w not in other_words_to_filter)
             for ingred_item in cleaned_ingred_list
             ]
    else:
        assert isinstance(cleaned_ingred_list[0], str)
        ingreds_only = [' '.join(w for w in ingred_item.split() if w not in meas_units and w not in preparatory_descriptions and w not in other_words_to_filter)
             for ingred_item in cleaned_ingred_list
             ]
    ingreds_only = filter(None, ingreds_only) # remove empty strings at end
    #print ingreds_only
    
    brand_names = [
        'a(r)',
        'accent(r)',
        'adams(r)',
        'al fresco(r)',
        'accent(r)',
        'annies lemon chive dressing(r)',
        'archer farms(tm)',
        'backdraft fire sauce(r)',
        'barilla(r)',
        'beau monde (tm)',
        'beechers(r) flagship', 'beechers(r)',
        'beer shiner(r)',
        'best foods(r)',
        'better than bouillon(r)',
        'birds eye(r)',
        'bisquick(r)',
        'bisquick (r)',
        'borden(r)',
        'bragg(r)',
        'briannas home style(r)', 'briannas(r)',
        'brooks(r)',
        'bruces(r)',
        'budweiser(r)',
        'buitoni(r)',
        'bumble bee(r)',
        'bushs best(r)', 'bushs homestyle(r)', 'bushs chili magic(r)', 'bushs(r)',
        'butterball(r)',
        'butter buds(r)',
        'campari(r)',
        'campbells(r) healthy request', 'campbells(r)',
        'carolina pride(r)',
        'carroll shelbys(r)',
        'catelli bistro(r)',
        'cavenders(r)',
        'chambord(r)',
        'chef paul prudhommes salmon magic(r)',
        'cheese whiz(r)', 'cheez whiz(r)',
        'cholula(r)',
        'clamato(r)',
        'classicmac(tm)',
        'classico (tm)',
        'classico (r)',
        'classico(r)',
        'claussen(r)',
        'coke(r)',
        'cointreau(r)',
        'college inn(r)',
        'contadina(r)',
        'cool whip(r)',
        'coors(r)',
        'cornnuts (r)',
        'corona(r)',
        'craisins(r)',
        'crazy steves(tm) cajun cukes', 'crazy steves(r)',
        'crisco(r)',
        'daisy(r)',
        'dei fratelli(r)',
        'del monte(r)',
        'diamond crystal(r)',
        'dickinsons(r)',
        'dole(r)',
        'dole asian island crunch(tm)',
        'dole(r) veggie(tm)',
        'doritos(r)',
        'dubliner(r)',
        'dukes(r)',
        'durkee(r)', 'famous sauce(r)',
        'eagle brand(r)',
        'earth balance(r)',
        'eckrich(r)',
        'edward sons(r)',
        'egg beaters(r)',
        'el pato(r)',
        'embasa(r)',
        'emerald(r)',
        'equal(r)',
        'farmland(r)',
        'fat tire(r)',
        'fiesta blend(r)',
        'follow your heart(r) veganaise(r)', 'follow your heart(r)',
        'foster farms(r)',
        'franks(r) redhot', 'franks red hot (r)', 'franks redhot (r)', 'franks redhot(r)', 'redhot(r)', 'franks(r)',
        'frenchs(r)',
        'frichik(r)',
        'fritos(r)',
        'frontera(r)',
        'goya(r)',
        'gebhardt(r)',
        'girards(r) olde venice',
        'good seasons(r)',
        'gourmet garden(tm)',
        'grapenuts(tm)',
        'great american spice co(tm)',
        'green giant(r)',
        'guinness(r)',
        'harp(r)',
        'hatch(r)',
        'healthy choice(r)',
        'heineken(r)', 
        'heinz (r)', 'heinz(r)',
        "hellmann's(r) best foods(r)", 'hellmanns(r) best foods(r)', 'hellmanns(r)best foods(r)',
        'hellmanns light(r)', 'hellmans(r) light', 'hellmanns(r) low fat',
        'hellmans(r)', "hellmann's(r)",
        'hidden valley ranch(r)', 'hidden valley(r) original ranch(r)',
        'hillshire farm(r)',
        'holland house(r)',
        'horizon(r)',
        'hormel(r)', 'hormel(tm)',
        'house tsang(r)',
        'hunts(r)',
        'idahoan(r)','baby reds(r)', 'buttery homestyle(r)',
        'imagine(r)', 'imagine(tm)',
        'jack daniels(r)',
        'jello(r)',
        'jennieo(r)',
        'jiffy(r)',
        'jim beam(r)',
        'jimmy dean(r)',
        'johnsonville(r)',
        'johnnys seasoning salt(r)',
        'kame(r)',
        'karys(r)',
        'kc masterpiece(r)',
        'keebler club(r)', 'keebler(r)',
        'keens(r)',
        'kens steak house(r)',
        'kens steak house lite northern italian dressing(r)',
        'kerrygold(r)',
        'kewpie(r)',
        'king arthur(r)',
        'king oscar(r)',
        'kirkland(r)',
        'kitchen basics(r)',
        'kitchen bouquet(r)',
        'kitchen secrets(r)',
        'klondike gourmet(r)', 'klondike goldust(r)', 'klondike(r)',
        'knorr(r) rice sides(tm)', 'knorr(r) fiesta sides(tm)', 'knorr(r)', 'knorrs(r)',
        'knox (r)',
        'kraft(r)',
        'laphroiag(r)',
        'laughing cow(r)',
        'lawrys(r)',
        'libbys(r)',
        'lightlife(r) organic smoky tempeh strips(r)', 'lightlife(r)',
        'lipton(r) recipe secrets(r)', 'lipton(r)',
        'locatelli(r)',
        'loma linda(r)',
        'lotus(r)',
        'louisiana(r)',
        'maggi(r)',
        'mahatma(r)',
        'maifun(r)',
        'maille(r)',
        'malibu(r)',
        'marie callenders(r)',
        'maries(r)',
        'market pantry(tm)',
        'marzettis(r)',
        'maui(r)',
        'mazola(r)',
        'mccormick chili seasoning mix sodium(r)', 'mccormick(r)',
        'mexene(r)',
        'mexenes(r)',
        'mexicorn(r)',
        'miller lite(r)',
        'mimiccreme(r)',
        'minute(r)',
        'miracle whip(r)',
        'miracle whip(tm)',
        'miracle whip (tm)',
        'miracle whip free(r)',
        'miracle whip light(r)', 'light miracle whip(r)',
        'montreal steak seasoning(r)',
        'morningstar farms(r) grillers recipe crumbles(r)', 'morningstar farms(r) recipe crumbles(r)', 'morningstar farms(r)',
        'morton(r) natures seasons(r)', 'morton(r)',
        'motts(r)',
        'mountain dew(r)',
        'mrs butterworths(r)',
        'mrs dash(r)',
        'muir glen(r)',
        'nabisco(r)',
        'natural goodness(tm)',
        'negra modelo(r)',
        'newcastle(r)',
        'newmans own(r)',
        'newmans own(r) lighten up(r)', 'lighten up(r)',
        'niblets(r)',
        'nissin(r) top ramen',
        'no yolks(r)',
        'ocean spray(r)',
        'old bay(r)', 'old bay (tm)', 'old bay(tm)',
        'old el paso(r)',
        'olive garden(r)',
        'oreida(r) steam n mash(r)', 'oreida(r)', 'ore ida(r)',
        'oreo(r)',
        'ortega(r)',
        'ottogi ramyonsari(r)', 'ottogi(r)',
        'quorn(tm)',
        'pace(r)',
        'pam(r)',
        'pataks(r)',
        'penzeys(r)', 'penzys(r)',
        'pepperidge farm(r)',
        'perdue(r)',
        'pernod(r)',
        'perrins(r)',
        'pillsbury grands(r)',
        'planters(r)',
        'plochmans(r)',
        'progresso(r)',
        'ragu(r) old world style(r)', 'ragu(r)',
        'rapunzel(r)',
        'ready rice(r)',
        'realime(r)',
        'red gold(r)',
        'renees(r)',
        'reynolds wrap(r)', 'reynolds(r)',
        'rice sides(tm)',
        'ronzoni(r)',
        'rotel(r)',
        'saco(r)',
        'sambazon',
        'samuel smith(r)',
        'sargento(r)',
        'sason accent(r)',
        'sb(r) golden curry', 'sb(r)',
        'seasonall(r)',
        'shiner bock(r)',
        'silk(r)',
        'simple crisp(tm)',
        'simply balanced(tm)',
        'simply potatoes(r)',
        'slap ya mama(r)',
        'smithfield(r)',
        'snack factory(r)',
        'snickers(r)',
        'so delicious(r)',
        'southern comfort (r)',
        #'spam(r)',
        'spectrum(r)',
        'spice islands(r)',
        'spike(r)',
        'splenda(r)',
        'sriracha(r)',
        'steam whistle pilsner(r)',
        'sunsweet(r) dnoir(tm)',
        'sutton dodge(tm)',
        'swanson(r) vegetable flavor boost(r)', 'swanson(r)', 'natural goodness(r)',
        'sweet baby rays(r)',
        'sweetn low(r)',
        'tabasco(r)', 'tabasco(tm)',
        'taco bell(r)',
        'tajin(r)',
        'tapatio(r)',
        'tater tots(r)',
        'texas pete(r)', 'texas petess(r)',
        'thai kitchen(r)',
        'three bridges(r)',
        'tony chacheres(r)',
        'top ramen(r)',
        'trader joes(r)',
        'truroots(r)',
        'truvia(r)',
        'uncle bens (r)', 'uncle bens(r)',
        'v(r)', # what remains from "V8"
        'valentina(r)',
        'vegenaise(r)',
        'veggie(tm)',
        #'velveeta(r)',
        'vidalia(r)',
        'vietti(r)',
        'vindaloo sauce maya kaimal(r)', 'vindaloo(r)',
        'voskos(r)',
        'wesson(r)',
        'white lily(r)',
        'wishbone(r)',
        'wondra(r)',
        'yoplait(r)',
        'yuengling(r)',
        'yves veggie cuisine(r)',
        'zatarains(r)',
        'zing zang(r)',
    ]
    
    def remove_brands(ingred_str):
        str_without_brand = ingred_str
        for bn in brand_names:
            str_without_brand = str_without_brand.replace(bn, '')
        return str_without_brand
    
    ingreds_without_brands = [' '.join(remove_brands(ingred_str).strip().split()) for ingred_str in ingreds_only]
    
    return ingreds_without_brands


In [3]:
def syntax_text(text):
    """Detects syntax in the text."""
    client = language.LanguageServiceClient()

    if isinstance(text, six.binary_type):
        text = text.decode('utf-8')

    # Instantiates a plain text document.
    document = types.Document(
        content=text,
        type=enums.Document.Type.PLAIN_TEXT)

    # Detects syntax in the document. You can also analyze HTML with:
    #   document.type == enums.Document.Type.HTML
    tokens = client.analyze_syntax(document).tokens

    # part-of-speech tags from enums.PartOfSpeech.Tag
    pos_tag = ('UNKNOWN', 'ADJ', 'ADP', 'ADV', 'CONJ', 'DET', 'NOUN', 'NUM',
               'PRON', 'PRT', 'PUNCT', 'VERB', 'X', 'AFFIX')

    token_pos_dict = OrderedDict()
    for token in tokens:
        #print(u'{}: {}'.format(pos_tag[token.part_of_speech.tag], token.text.content))
        if token.text.content not in token_pos_dict:
            token_pos_dict[token.text.content] = pos_tag[token.part_of_speech.tag]
        else:
            #print "{} token already in dict".format(token.text.content)
            pass
    return token_pos_dict

In [4]:
def pos_dict_to_str(d):
    # Takes an OrderedDict that maps each ingredient to a POS
    # Returns a string containing only ingredient words that are nouns or adjectives
    pos_tags_to_remove = ('UNKNOWN', 'ADP', 'ADV', 'CONJ', 'DET', 'NUM', 'PRON', 'PRT', 'PUNCT', 'VERB', 'X', 'AFFIX')
    # keep only nouns and adjectives
    #d_keep = {k:v for k, v in d.iteritems() if v not in pos_tags_to_remove}
    d_keep = OrderedDict((k, v) for k, v in d.iteritems() if v not in pos_tags_to_remove)

    #     for k, v in d_keep.iteritems():
    #         print "word: {} -> tag: {}".format(k, v)

    keep_str = ' '.join('{}'.format(key) for key, val in d_keep.iteritems())
    #print keep_str
    return keep_str

In [5]:
df = pd.read_pickle('CleanedIngredients.pkl')
print len(df)

In [6]:
pd.set_option('display.max_colwidth', -1)

In [7]:
df.sample(5)

,categories,cookingTime,description,ingredients,instructionSteps,name,rating,ratingCount,url,cookingTimeMinutes,cleanedIngredients
317,"[Chicken Breasts, Cheesy Chicken Breasts, Baked Chicken Breasts, Baked and Roasted Chicken, Everyday Cooking, Main Dishes, Recipes, Provolone Cheese, Spring Main Dishes, Quick and Easy Chicken Dinners]",PT55M,"Very flavorful chicken and asparagus dish with almost a hollandaise-inspired flavor with a little crunch from the Panko breadcrumbs. Quick and easy to prepare, the presentation is simple yet elegant. Great served with artichokes!","[1/2 cup mayonnaise, 3 tablespoons Dijon mustard, 1 lemon, juiced and zested, 2 teaspoons dried tarragon, 1 teaspoon ground black pepper, 1/2 teaspoon salt, 16 spears fresh asparagus, trimmed, 4 skinless, boneless chicken breast halves, 4 slices provolone cheese, 1 cup panko bread crumbs, Add all ingredients to list, Add all ingredients to list]","[Preheat oven to 475 degrees F (245 degrees C). Grease a baking dish. In a bowl, mix together the mayonnaise, Dijon mustard, lemon juice, lemon zest, tarragon, salt, and pepper until the mixture is well combined. Set aside., Cook asparagus in the microwave on High until bright green and just tender, 1 to 1 1/2 minutes. Set the asparagus spears aside. Place a chicken breast between two sheets of heavy plastic (resealable freezer bags work well) on a solid, level surface. Firmly pound the chicken breast with the smooth side of a meat mallet to a thickness of about 1/4 inch. Repeat with the rest of the chicken breasts., Place 1 slice of provolone on each chicken breast, and top the cheese with 4 asparagus spears per breast. Roll the chicken breasts around the asparagus and cheese, making a tidy package, and place, seam sides down, in the prepared baking dish. With a pastry brush, apply a coating of the mayonnaise mixture to each chicken breast, and sprinkle each with panko crumbs, pressing the crumbs into the chicken to make a coating., Bake in the preheated oven until the crumbs are browned and the chicken juices run clear, about 25 minutes.]",Chicken Asparagus Roll-Ups,4.62,591,http://allrecipes.com/recipe/213645/chicken-asparagus-roll-ups/,55.0,"[mayonnaise, dijon mustard, lemon, tarragon, black pepper, salt, asparagus, skinless boneless chicken breast, provolone cheese, panko bread crumbs]"
11148,"[Reddi-wip®, Trusted Brands, Fruit Dips, Cheese Dips and Spreads, Dips and Spreads, Cheese Appetizers, Appetizers and Snacks, Everyday Cooking, Recipes, Cantaloupe Recipes]",PT15M,NaN,"[4 ounces cream cheese, 1 (3.25 ounce) cup Snack Pack® Vanilla Pudding, 1 3/4 cups Reddi-wip® Original Dairy Whipped Topping, 2 tablespoons confectioners' sugar, 2 cups assorted fruit pieces for dipping (strawberries, grapes, cantaloupe, etc.), Add all ingredients to list, Add all ingredients to list]","[Place cream cheese in medium microwave-safe bowl; cover. Microwave on HIGH 30 seconds to soften. Add pudding; whisk until smooth., Add Reddi-wip and sugar; whisk together until blended. Serve dip with fruit.]",Cheesecake Fruit Dip from Reddi-wip®,0.00,0,http://allrecipes.com/recipe/254283/cheesecake-fruit-dip-from-reddi-wip/,15.0,"[cream cheese, snack pack(r) vanilla pudding, reddiwip(r) dairy whipped topping, confectioners sugar, assorted fruit dipping strawberries grapes cantaloupe etc]"
4092,"[Rye Bread, Whole Grain Bread, Yeast Bread, Bread Machine Rye Bread, Bread Machine, Bread Recipes, Recipes, Caraway, Oktoberfest Recipes, Superfoods - Whole Grains]",PT3H40M,NaN,"[1 1/4 cups warm water (110 degrees F/45 degrees C), 1 tablespoon vegetable oil, 1 1/2 tablespoons molasses, 1 1/2 cups unbleached bread flour, 1 cup rye flour, 1/2 cup whole wheat flour, 1/4 cup vital wheat gluten, 1 teaspoon salt, 3 tablespoons dry milk powder, 2 teaspoons instant coffee powder, 2 tablespoons unsweetened cocoa powder, 1 tablespoon caraway seed, 1 1/2 teaspoons active dry yeast, Add all ingredients to list, Add all ingredients to list]","[Place ingredients 